In [3]:
from bankmodelfactory.data.split import split_dataset
from bankmodelfactory.data.load import SourceDataLoader
from bankmodelfactory.data.split import split_dataset, split_train_valid

loader = SourceDataLoader()
data = loader.load_dataset()
summary = loader.inspect_dataset()

display(summary)

X_train, X_test, y_train, y_test = split_dataset(data)


2025-10-31 17:40:50 | INFO     | bankmodelfactory.data.load:__init__:53 - [SourceDataLoader] Configuration loaded from: /Users/surelmanda/BankModelFactory/configs/data.yaml
2025-10-31 17:40:50 | SUCCESS  | bankmodelfactory.data.load:load_dataset:85 - [SourceDataLoader] 'Bank Marketing Dataset' loaded successfully from: /Users/surelmanda/BankModelFactory/data/raw/bank-full.csv
2025-10-31 17:40:50 | INFO     | bankmodelfactory.data.load:load_dataset:86 - [SourceDataLoader] Shape: 45211 rows × 17 columns
2025-10-31 17:40:50 | INFO     | bankmodelfactory.data.load:inspect_dataset:124 - [SourceDataLoader] Inspecting dataset structure...


,Column,Type,Missing,% Missing,Cardinality,Examples
0,age,int64,0,0.0,77,"[58, 44, 33, 47, 35]"
9,day,int64,0,0.0,31,"[5, 6, 7, 8, 9]"
15,poutcome,object,0,0.0,4,"[unknown, failure, other, success]"
14,previous,int64,0,0.0,41,"[0, 3, 1, 4, 2]"
13,pdays,int64,0,0.0,559,"[-1, 151, 166, 91, 86]"
12,campaign,int64,0,0.0,48,"[1, 2, 3, 5, 4]"
11,duration,int64,0,0.0,1573,"[261, 151, 76, 92, 198]"
10,month,object,0,0.0,12,"[may, jun, jul, aug, oct, nov, dec, jan, feb, ..."
8,contact,object,0,0.0,3,"[unknown, cellular, telephone]"
1,job,object,0,0.0,12,"[management, technician, entrepreneur, blue-co..."


2025-10-31 17:40:51 | INFO     | bankmodelfactory.data.split:split_dataset:67 - [Dataset Split] YAML config → test_size=0.1, stratify=True, random_state=42
2025-10-31 17:40:51 | SUCCESS  | bankmodelfactory.data.split:split_dataset:85 - [Dataset Split] Completed. Train shape=(40689, 16), Test shape=(4522, 16)


In [4]:
from bankmodelfactory.features import FeaturePipeline

pipeline = FeaturePipeline()

display(pipeline.diagram())

X_train_ready, y_train_ready = pipeline.fit_transform(X_train, y_train)

X_test_ready, y_test_ready = pipeline.transform(X_test, y_test)

,steps,"[('semantic_cleaner', ...), ('behavioral_transformer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False


2025-10-31 17:40:54 | INFO     | bankmodelfactory.features.FeaturePipeline:fit_transform:124 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-31 17:40:54 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-31 17:40:54 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-31 17:40:54 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-31 17:40:54 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-31 17:40:54 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-31 17:40:54 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [BehavioralTransformer] Generated features. Shape

In [9]:
import numpy as np
X_train_ready = X_train_ready.select_dtypes(include=[np.number])
X_test_ready = X_test_ready.select_dtypes(include=[np.number])


In [7]:
# 2. Split du train pour validation interne (val_size et stratify pris dans le YAML)
X_train_final, y_train_final, X_valid, y_valid = split_train_valid(X_train_ready, y_train_ready,shuffle=True,random_state=42)

print("Train final :", X_train_final.shape)
print("Validation  :", X_valid.shape)
print("Test        :", X_test.shape)


2025-10-31 17:41:09 | INFO     | bankmodelfactory.data.split:split_train_valid:153 - [split_train_valid] Completed. Total=40689, Train=32551 (80.0%), Valid=8138 (20.0%), Stratify=True, Params={'shuffle': True, 'random_state': 42}
Train final : (32551, 27)
Validation  : (8138, 27)
Test        : (4522, 16)


In [11]:
from torch.utils.data import DataLoader
from bankmodelfactory.utils.preview import preview_dataloader
from bankmodelfactory.data.bank_dataset import BankMarketingDataset

# ============================================================
# 1. TRAIN DATASET
# ============================================================
print("\n" + "=" * 80)
print("[1/3] Initializing TRAIN DataLoader")
print("=" * 80 + "\n")

train_dataset = BankMarketingDataset(X_train_ready, y_train_ready)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

preview_dataloader("TRAIN", train_loader, n_batches=2)


# ============================================================
# 2. VALIDATION DATASET
# ============================================================
print("\n" + "=" * 80)
print("[2/3] Initializing VALIDATION DataLoader")
print("=" * 80 + "\n")

valid_dataset = BankMarketingDataset(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

preview_dataloader("VALIDATION", valid_loader, n_batches=2)


# ============================================================
# 3. TEST DATASET
# ============================================================
print("\n" + "=" * 80)
print("[3/3] Initializing TEST DataLoader")
print("=" * 80 + "\n")

test_dataset = BankMarketingDataset(X_test_ready, y_test_ready)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

preview_dataloader("TEST", test_loader, n_batches=2)


# ============================================================
# SUMMARY
# ============================================================
print("\n" + "=" * 80)
print("All DataLoaders have been successfully initialized and previewed.")
print("=" * 80 + "\n")



[1/3] Initializing TRAIN DataLoader

2025-10-31 17:48:37 | INFO     | bankmodelfactory.utils.device:get_device:39 - [DeviceManager] Using device: mps
2025-10-31 17:48:37 | INFO     | bankmodelfactory.data.bank_dataset:__init__:85 - [BankMarketingDataset] Initialized with 40689 samples and 27 features on device='mps'.

TRAIN DataLoader Preview

Batch 0
Features: shape = (128, 27)
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ...
--------------------------------------------------
Batch 1
Features: shape = (128, 27)
Labels: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ...
--------------------------------------------------

✔ Preview completed for TRAIN DataLoader

2025-10-31 17:48:38 | INFO     | bankmodelfactory.utils.preview:preview_dataloader:64 - [preview_dataloader] Displayed 2 batches from 'TRAIN'.

[2/3] Initializing VALIDATION DataLoader

2025-10-31 17:48:38 | INFO     | bankmodelfactory.utils.device:get_device:39 - [DeviceManager] Using device: mps
2025-10-31 

In [ ]:
from bankmodelfactory.features import FeaturePipeline

pipeline = FeaturePipeline()  # target_strategy="auto" par défaut
X_train_ready, y_train_ready = pipeline.fit_transform(X_train, y_train)
X_test_ready, y_test_ready = pipeline.transform(X_test, y_test)

# vérifications rapides
assert y_train_ready is not None
assert y_test_ready is not None


2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.FeaturePipeline:fit_transform:120 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.


2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 14:01:40 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 14:01:40 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [BehavioralTransformer] Generated features. Shape: (40689, 29)
2025-10-30 14:01:40 | INFO     | bankmodelfactory.features.business_encoder:fit:43 - [BusinessEncoder] Learned scaling for 17 numeric features.
2025-10-30 14:01:40 | SUCCESS  | bankmodelfactory.features.business_encoder:transform:71 - [BusinessEncoder] Encoding com

In [ ]:
FeaturePipeline()

,target_strategy,'auto'
,target_mapping,None


In [ ]:
from sklearn import set_config
from sklearn.pipeline import Pipeline
from bankmodelfactory.features.semantic_cleaner import SemanticCleaner
from bankmodelfactory.features.behavioral_transformer import BehavioralTransformer
from bankmodelfactory.features.business_encoder import BusinessEncoder

set_config(display='diagram')  # important pour l’affichage graphique dans Jupyter

visual = Pipeline([
    ("semantic_cleaner", SemanticCleaner()),
    ("behavioral_transformer", BehavioralTransformer()),
    ("business_encoder", BusinessEncoder()),
])
visual  # affiche le diagramme des étapes dans Jupyter



,steps,"[('semantic_cleaner', ...), ('behavioral_transformer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False


In [ ]:
from bankmodelfactory.features import FeaturePipeline

pipeline = FeaturePipeline()
X_train_ready, y_train_ready = pipeline.fit_transform(X_train, y_train)

X_test_ready, y_test_ready = pipeline.transform(X_test, y_test)

pipeline.diagram()



2025-10-30 14:10:24 | INFO     | bankmodelfactory.features.FeaturePipeline:fit_transform:124 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 14:10:24 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 14:10:24 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 14:10:24 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-30 14:10:24 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 14:10:24 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 14:10:24 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [BehavioralTransformer] Generated features. Shape

,steps,"[('semantic_cleaner', ...), ('behavioral_transformer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False


In [ ]:
y_train_ready

array([0, 1, 0, ..., 0, 1, 0], shape=(40689,))

In [ ]:
from bankmodelfactory.features import FeaturePipeline
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# ==========================================================
# 1. Split the dataset (if not already done)
# ==========================================================

# If you already have X_train / X_test, skip this part.
# Otherwise, split from X_raw, y_raw for demonstration:
# X_train, X_test, y_train, y_test = train_test_split(
#     X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
# )

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# ==========================================================
# 2. Apply the full FeaturePipeline
# ==========================================================

pipeline = FeaturePipeline()

# Fit on training data and transform both train and test sets
X_train_ready, y_train_ready = pipeline.fit_transform(X_train, y_train)
X_test_ready, y_test_ready = pipeline.transform(X_test), y_test  # No refitting

# ==========================================================
# 3. Diagnostics
# ==========================================================

print("FeaturePipeline applied successfully.")
print(f"Train shape: {X_train_ready.shape}, Test shape: {X_test_ready.shape}")
print("Number of columns:", len(X_train_ready.columns))
print("Any NaN in train:", X_train_ready.isna().any().any())
print("Any NaN in test:", X_test_ready.isna().any().any())

# Verify that train and test have identical columns
if list(X_train_ready.columns) == list(X_test_ready.columns):
    print("Columns match between train and test.")
else:
    print("Columns mismatch detected.")
    diff = set(X_train_ready.columns) ^ set(X_test_ready.columns)
    print("Difference:", diff)

# Show a small preview of encoded features
print("\nSample of encoded features:")
print(X_train_ready.head(3))

# ==========================================================
# 4. Save processed data for modeling
# ==========================================================

X_train_ready.to_csv("/Users/surelmanda/BankModelFactory/data/processed/X_train_ready.csv", index=False)
X_test_ready.to_csv("/Users/surelmanda/BankModelFactory/data/processed/X_test_ready.csv", index=False)

print("Processed datasets saved to data/processed/")


Training set: (40689, 16), Test set: (4522, 16)
2025-10-30 13:56:54 | INFO     | bankmodelfactory.features.FeaturePipeline:fit_transform:95 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 13:56:54 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:56:54 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:56:55 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-30 13:56:55 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 13:56:55 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 13:56:55 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [B

In [ ]:
y_train

36115     no
33881    yes
12044     no
36016     no
26354     no
        ... 
8884     yes
16952     no
34781     no
43825    yes
30211     no
Name: y, Length: 40689, dtype: object

In [ ]:
from bankmodelfactory.features import FeaturePipeline

pipeline = FeaturePipeline()
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)



2025-10-30 13:39:16 | INFO     | bankmodelfactory.features.pipeline:fit_transform:28 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 13:39:16 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:39:16 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:39:16 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-30 13:39:16 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 13:39:16 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 13:39:16 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [BehavioralTransformer] Generated features. Shape: (40689

In [ ]:
from bankmodelfactory.features import FeaturePipeline


# Apply full business-driven preprocessing
pipeline = FeaturePipeline()
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)



2025-10-30 13:38:07 | INFO     | bankmodelfactory.features.pipeline:fit_transform:28 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 13:38:07 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:38:07 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:38:07 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
2025-10-30 13:38:07 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 13:38:07 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 13:38:07 | SUCCESS  | bankmodelfactory.features.behavioral_transformer:transform:75 - [BehavioralTransformer] Generated features. Shape: (40689

In [ ]:
from bankmodelfactory.features.semantic_cleaner import SemanticCleaner

cleaner = SemanticCleaner()
X_clean = cleaner.fit_transform(X_train.copy())



print("Before --")
print("Missing values per column:\n", X_train.isna().sum())

print("SemanticCleaner OK")
print("Shape:", X_clean.shape)
print("Missing values per column:\n", X_clean.isna().sum())
print("Numeric columns:", X_clean.select_dtypes('number').columns.tolist())


2025-10-30 13:28:34 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:28:34 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:28:34 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
Before --
Missing values per column:
 age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64
SemanticCleaner OK
Shape: (40689, 21)
Missing values per column:
 age            0
job            0
marital        0
education      0
default        0
balance        0
housing        0
loan           0
contact        0
day            0
month          0
duration       0
campaign       0
pdays     

In [ ]:
from bankmodelfactory.features.semantic_cleaner import SemanticCleaner
from bankmodelfactory.features.behavioral_transformer import BehavioralTransformer
import pandas as pd

# Étape 1 — Semantic Cleaning
cleaner = SemanticCleaner()
X_clean = cleaner.fit_transform(X_train.copy())

print("🧹 After SemanticCleaner")
print("----------------------")
print("Shape:", X_clean.shape)
print("Missing values per column:\n", X_clean.isna().sum()[X_clean.isna().sum() > 0])
print("Numeric columns:", X_clean.select_dtypes("number").columns.tolist())

# Étape 2 — Behavioral Transformation
behavior = BehavioralTransformer()
X_behavior = behavior.fit_transform(X_clean.copy())

print("\n🧠 After BehavioralTransformer")
print("-----------------------------")
print("Shape:", X_behavior.shape)
print("Missing values per column:\n", X_behavior.isna().sum()[X_behavior.isna().sum() > 0])
print("Numeric columns:", X_behavior.select_dtypes("number").columns.tolist())

# Focus sur les nouvelles colonnes générées
new_cols = set(X_behavior.columns) - set(X_clean.columns)
print("\n🆕 New features added:", new_cols)


2025-10-30 13:38:12 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:38:12 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:38:12 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
🧹 After SemanticCleaner
----------------------
Shape: (40689, 21)
Missing values per column:
 Series([], dtype: int64)
Numeric columns: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'has_housing', 'has_loan', 'has_default', 'is_summer', 'is_winter']
2025-10-30 13:38:12 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 13:38:12 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 13:38:12 | SU

In [ ]:
from bankmodelfactory.features.semantic_cleaner import SemanticCleaner
from bankmodelfactory.features.behavioral_transformer import BehavioralTransformer
from bankmodelfactory.features.business_encoder import BusinessEncoder
import pandas as pd
import numpy as np

# ======================================================
# Étape 1 — Semantic Cleaning
# ======================================================
cleaner = SemanticCleaner()
X_clean = cleaner.fit_transform(X_train.copy())

print("🧹 After SemanticCleaner")
print("----------------------")
print("Shape:", X_clean.shape)
print("Missing values per column:\n", X_clean.isna().sum()[X_clean.isna().sum() > 0])
print("Numeric columns:", X_clean.select_dtypes("number").columns.tolist())


# ======================================================
# Étape 2 — Behavioral Transformation
# ======================================================
behavior = BehavioralTransformer()
X_behavior = behavior.fit_transform(X_clean.copy())

print("\n🧠 After BehavioralTransformer")
print("-----------------------------")
print("Shape:", X_behavior.shape)
print("Missing values per column:\n", X_behavior.isna().sum()[X_behavior.isna().sum() > 0])
print("Numeric columns:", X_behavior.select_dtypes("number").columns.tolist())

# Focus sur les nouvelles colonnes générées
new_cols_behavior = set(X_behavior.columns) - set(X_clean.columns)
print("\n🆕 New features added:", new_cols_behavior)


# ======================================================
# Étape 3 — Business Encoder
# ======================================================
encoder = BusinessEncoder()
encoder.fit(X_behavior)  # Fit sur les données transformées
X_encoded = encoder.transform(X_behavior)

print("\n💼 After BusinessEncoder")
print("-------------------------")
print("Shape:", X_encoded.shape)

# Vérification des NaN
missing = X_encoded.isna().sum()
if missing.sum() == 0:
    print("✅ No missing values after encoding.")
else:
    print("⚠️ Missing values found:\n", missing[missing > 0])

# Liste des nouvelles features générées par l’encodage
new_cols_encoder = set(X_encoded.columns) - set(X_behavior.columns)
print("\n🆕 New features added by BusinessEncoder:")
for col in sorted(new_cols_encoder):
    print("   •", col)

# Vérification des valeurs numériques et du scaling
num_cols = X_encoded.select_dtypes(include=np.number).columns.tolist()
print("\n📊 Numeric columns after encoding:", num_cols[:10], "...")
print("Total numeric columns:", len(num_cols))

# Aperçu des statistiques pour vérifier le scaling
scaled_preview = X_encoded[num_cols].describe().loc[["mean", "std"]]
print("\n📈 Scaled feature stats (mean ~ 0, std ~ 1):")
print(scaled_preview.transpose().head(10))

# ======================================================
# Résumé
# ======================================================
print("\n✅ Partial pipeline test complete!")
print(f"Input shape: {X_train.shape} → Final shape: {X_encoded.shape}")
print(f"Total new columns (behavior + encoder): {len(new_cols_behavior | new_cols_encoder)}")


2025-10-30 13:38:15 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:26 - [SemanticCleaner] Fitting stage started.
2025-10-30 13:38:15 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:30 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 13:38:15 | SUCCESS  | bankmodelfactory.features.semantic_cleaner:transform:66 - [SemanticCleaner] Cleaning complete. Shape: (40689, 21)
🧹 After SemanticCleaner
----------------------
Shape: (40689, 21)
Missing values per column:
 Series([], dtype: int64)
Numeric columns: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'has_housing', 'has_loan', 'has_default', 'is_summer', 'is_winter']
2025-10-30 13:38:15 | INFO     | bankmodelfactory.features.behavioral_transformer:fit:27 - [BehavioralTransformer] Fitting stage started.
2025-10-30 13:38:15 | INFO     | bankmodelfactory.features.behavioral_transformer:transform:31 - [BehavioralTransformer] Creating business features...
2025-10-30 13:38:15 | SU

In [ ]:
from bankmodelfactory.features.semantic_cleaner import SemanticCleaner
from bankmodelfactory.features.behavioral_transformer import BehavioralTransformer
from bankmodelfactory.features.business_encoder import BusinessEncoder
from bankmodelfactory.utils.logger import get_logger
import pandas as pd
import numpy as np

logger = get_logger(__name__)

# ======================================================
# Étape 1 — Semantic Cleaning
# ======================================================
cleaner = SemanticCleaner()
X_clean = cleaner.fit_transform(X_train.copy())

print("🧹 After SemanticCleaner")
print("----------------------")
print("Shape:", X_clean.shape)
print("Missing values per column:\n", X_clean.isna().sum()[X_clean.isna().sum() > 0])
print("Numeric columns:", X_clean.select_dtypes("number").columns.tolist())


# ======================================================
# Étape 2 — Behavioral Transformation
# ======================================================
behavior = BehavioralTransformer()
X_behavior = behavior.fit_transform(X_clean.copy())

print("\n🧠 After BehavioralTransformer")
print("-----------------------------")
print("Shape:", X_behavior.shape)
print("Missing values per column:\n", X_behavior.isna().sum()[X_behavior.isna().sum() > 0])
print("Numeric columns:", X_behavior.select_dtypes("number").columns.tolist())

# Focus sur les nouvelles colonnes générées
new_cols_behavior = set(X_behavior.columns) - set(X_clean.columns)
print("\n🆕 New features added:", new_cols_behavior)


# ======================================================
# Étape 3 — Business Encoder
# ======================================================
encoder = BusinessEncoder()
encoder.fit(X_behavior)  # Fit sur les données après transformation
X_encoded = encoder.transform(X_behavior)

print("\n💼 After BusinessEncoder")
print("-------------------------")
print("Shape:", X_encoded.shape)

# Vérification des NaN
missing = X_encoded.isna().sum()
if missing.sum() == 0:
    print("✅ No missing values after encoding.")
else:
    print("⚠️ Missing values found:\n", missing[missing > 0])

# Liste des nouvelles features générées par l’encodage
new_cols_encoder = set(X_encoded.columns) - set(X_behavior.columns)
print("\n🆕 New features added by BusinessEncoder:")
for col in sorted(new_cols_encoder):
    print("   •", col)

# Vérification des valeurs numériques et du scaling
num_cols = X_encoded.select_dtypes(include=np.number).columns.tolist()
print("\n📊 Numeric columns after encoding:", num_cols[:10], "...")
print("Total numeric columns:", len(num_cols))

# Aperçu des statistiques pour vérifier le scaling
scaled_preview = X_encoded[num_cols].describe().loc[["mean", "std"]]
print("\n📈 Scaled feature stats (mean ~ 0, std ~ 1):")
print(scaled_preview.transpose().head(10))

# ======================================================
# Résumé
# ======================================================
print("\n✅ Pipeline partial test complete!")
print(f"Input shape: {X_train.shape} → Final shape: {X_encoded.shape}")
print(f"Total new columns (behavior + encoder): {len(new_cols_behavior | new_cols_encoder)}")


TypeError: get_logger() takes 0 positional arguments but 1 was given

In [ ]:
from bankmodelfactory.features import FeaturePipeline

pipeline = FeaturePipeline()
print(type(pipeline))


<class 'bankmodelfactory.features.pipeline.FeaturePipeline'>


In [ ]:
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)

2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.pipeline:fit_transform:48 - [FeaturePipeline] Starting full preprocessing pipeline...
2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.semantic_cleaner:fit:39 - [SemanticCleaner] Fitting stage started.
2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:43 - [SemanticCleaner] Starting semantic cleaning...
2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:49 - [SemanticCleaner] Harmonized string columns: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:61 - [SemanticCleaner] Created boolean indicators: has_housing, has_loan, has_default
2025-10-30 12:55:45 | INFO     | bankmodelfactory.features.semantic_cleaner:transform:67 - [SemanticCleaner] Added features: is_summer, is_winter
2025-10-30 12:55:45 | SUCCESS  | bankmodel

/Users/surelmanda/BankModelFactory/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/surelmanda/BankModelFactory/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/surelmanda/BankModelFactory/.venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
X_ready

,num__age,num__balance,num__day,num__month,num__duration,num__campaign,num__pdays,num__previous,num__has_housing,num__has_loan,...,cat__contact_cellular,cat__contact_telephone,cat__contact_nan,cat__poutcome_failure,cat__poutcome_other,cat__poutcome_success,cat__poutcome_nan,cat__age_group_adult,cat__age_group_senior,cat__age_group_young
0,0.480194,-0.558772,-0.577600,NaN,-0.296511,-0.247617,-0.410592,-0.246905,0.892470,2.285698,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,-0.273891,-0.447569,1.706162,NaN,1.203971,-0.569306,-0.410592,-0.246905,0.892470,-0.437503,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-0.839455,0.144205,0.504182,NaN,-0.716955,0.074072,-0.410592,-0.246905,0.892470,-0.437503,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.480194,0.145189,-0.577600,NaN,-0.759385,0.074072,2.980627,0.605886,0.892470,-0.437503,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.140019,-0.447569,0.504182,NaN,1.007250,-0.247617,-0.410592,-0.246905,-1.120486,-0.437503,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40684,-1.310758,-0.310122,-1.418986,NaN,3.556913,-0.247617,-0.410592,-0.246905,0.892470,-0.437503,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
40685,0.385934,-0.421654,1.105172,NaN,-0.620523,-0.569306,-0.410592,-0.246905,0.892470,-0.437503,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
40686,-1.310758,0.391871,-1.178590,NaN,-0.122934,-0.569306,-0.410592,-0.246905,0.892470,-0.437503,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
40687,-1.687801,-0.145121,1.465766,NaN,1.076681,-0.247617,1.434630,3.590653,-1.120486,-0.437503,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from .pipeline import FeaturePipeline





pipeline = FeaturePipeline()
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)

ImportError: attempted relative import with no known parent package

In [ ]:
display(X_train)


display(y_train)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
36115,46,blue-collar,married,primary,no,-339,yes,yes,cellular,11,may,182,2,-1,0,unknown
33881,38,management,married,tertiary,no,0,yes,no,cellular,30,apr,571,1,-1,0,unknown
12044,32,management,single,tertiary,no,1804,yes,no,unknown,20,jun,73,3,-1,0,unknown
36016,46,admin.,divorced,primary,no,1807,yes,no,cellular,11,may,62,3,339,2,failure
26354,53,entrepreneur,married,tertiary,no,0,no,no,cellular,20,nov,520,2,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8884,27,blue-collar,married,secondary,no,419,yes,no,unknown,4,jun,1181,2,-1,0,unknown
16952,45,blue-collar,married,secondary,no,79,yes,no,cellular,25,jul,98,1,-1,0,unknown
34781,27,management,single,tertiary,no,2559,yes,no,cellular,6,may,227,1,-1,0,unknown
43825,23,student,single,secondary,no,922,no,no,cellular,28,may,538,2,184,9,failure


36115     no
33881    yes
12044     no
36016     no
26354     no
        ... 
8884     yes
16952     no
34781     no
43825    yes
30211     no
Name: y, Length: 40689, dtype: object

In [ ]:
from bankmodelfactory.features import FeaturePipeline



pipeline = FeaturePipeline()
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)

logger.success("[Main] Data ready for model training.")



TypeError: get_logger() takes 0 positional arguments but 1 was given

In [ ]:
from bankmodelfactory.data.load import SourceDataLoader
from bankmodelfactory.features import FeaturePipeline
from bankmodelfactory.utils.logger import get_logger

logger = get_logger(__name__)

logger.info("[Main] Starting BankModelFactory pipeline...")
loader = SourceDataLoader(config_path="configs/data.yaml")
X_train, y_train = loader.load_dataset()

pipeline = FeaturePipeline()
X_ready, y_ready = pipeline.fit_transform(X_train, y_train)

logger.success("[Main] Data ready for model training.")



TypeError: get_logger() takes 0 positional arguments but 1 was given

In [ ]:

from bankmodelfactory.features import FeaturePipeline

# Build and fit pipeline
pipeline = FeaturePipeline()
X_train_ready, y_train_ready = pipeline.fit_transform(X_train, y_train)




2025-10-30 12:25:53 | INFO     | bankmodelfactory.features.transformer:__init__:24 - [FeatureTransformer] Initialized.
2025-10-30 12:25:53 | INFO     | bankmodelfactory.features.pipeline:__init__:31 - [FeaturePipeline] Initialized successfully.
2025-10-30 12:25:53 | INFO     | bankmodelfactory.features.pipeline:fit_transform:37 - [FeaturePipeline] Starting fit_transform on training data...


TypeError: Cannot setitem on a Categorical with a new category (unknown), set the categories first

In [ ]:
from bankmodelfactory.utils.config import Config
from bankmodelfactory.utils.path import get_project_root

root = get_project_root()


# Load YAML configuration
cfg = Config.load({
    "data": f"/{get_project_root()}/configs/data.yaml"
})

# Access parameters
print(cfg.data["local_raw"])
print(cfg.data["test_size"])

/Users/surelmanda/BankModelFactory/data/raw/bank-full.csv
0.2


In [ ]:
X_train["age"].isna().sum(), X_test["age"].isna().sum()


(np.int64(0), np.int64(0))

In [ ]:
import sys
from pathlib import Path

def add_project_root():
    """Add project root (where pyproject.toml is located) to sys.path."""
    root = Path().resolve().parents[0]  # go back from utils/ → bankmodelfactory/ → src/
    sys.path.append(str(root))



add_project_root()

print(sys.path[-1])  # Vérifie que la racine du projet a bien été ajoutée

/Users/surelmanda/BankModelFactory


In [ ]:
from bankmodelfactory.utils.config import Config

In [ ]:
Path().resolve().parents[0]

PosixPath('/Users/surelmanda/BankModelFactory')

In [ ]:
pwd

'/Users/surelmanda/BankModelFactory/notebooks'

In [ ]:
from bankmodelfactory.utils.path import add_project_root

add_project_root()

from bankmodelfactory.utils.config import Config


ModuleNotFoundError: No module named 'bankmodelfactory'

In [ ]:
from bankmodelfactory.utils.config import Config


ModuleNotFoundError: No module named 'bankmodelfactory'

In [ ]:
import sys
sys.path.append("src")

from bankmodelfactory.utils.config import Config


ModuleNotFoundError: No module named 'bankmodelfactory'

In [ ]:
from bankmodelfactory.utils.config import Config

# Load YAML configuration
cfg = Config.load({
    "data": "configs/data.yaml"
})

# Access parameters
print(cfg.data["local_raw"])
print(cfg.data["test_size"])


ModuleNotFoundError: No module named 'bankmodelfactory'

In [ ]:
pwd

'/Users/surelmanda/BankModelFactory/notebooks'

In [ ]:
import os, sys
sys.path.append(os.path.abspath("src"))

from bankmodelfactory.utils.config import Config


ModuleNotFoundError: No module named 'bankmodelfactory'

In [ ]:
import os, sys
sys.path.append(os.path.abspath("src"))

print(sys.path[-1])  # Vérifie que "src" a bien été ajouté

from src.bankmodelfactory.utils.config import Config

cfg = Config.load({"data": "configs/data.yaml"})
print(cfg.data.keys())


/Users/surelmanda/BankModelFactory/notebooks/src


ModuleNotFoundError: No module named 'src'

In [ ]:
import os, sys
sys.path.append(os.path.abspath("src"))

print(sys.path[-1])  # Vérifie que "src" a bien été ajouté

/Users/surelmanda/BankModelFactory/notebooks/src


In [ ]:
print(sys.path)

['/Users/surelmanda/.pyenv/versions/3.11.6/lib/python311.zip', '/Users/surelmanda/.pyenv/versions/3.11.6/lib/python3.11', '/Users/surelmanda/.pyenv/versions/3.11.6/lib/python3.11/lib-dynload', '', '/Users/surelmanda/BankModelFactory/.venv/lib/python3.11/site-packages', '/Users/surelmanda/BankModelFactory/notebooks/src', '/Users/surelmanda/BankModelFactory/notebooks/src']
